In [1]:
import re
import pandas as pd

# ───────────────────────── Парсер ──────────────────────────
def parse_smart_log(path_or_str, top_n=15):
    """Принимает путь к .txt‑файлу или сам текст лога, 
       отдаёт DataFrame со шагами, dev/test/gap и всеми гиперпараметрами."""
    
    # читаем либо из файла, либо из уже переданной строки
    if '\n' in path_or_str or 'Шаг' in path_or_str:
        lines = path_or_str.splitlines()
    else:
        with open(path_or_str, encoding='utf-8') as f:
            lines = f.readlines()

    rows, current = [], {}

    step_re = re.compile(
        r"Шаг\s+(\d+):\s*([^=]+?)=\s*\((.*?)\)"
    )
    mean_re = re.compile(r"MEAN\s*=\s*([0-9.]+)")
    gap_re  = re.compile(r"GAP\s*=\s*([+-]?[0-9.]+)")

    for i, raw in enumerate(lines):
        line = raw.rstrip("\n")
        
        # ── 1. ищем строку «Шаг N: …»  ───────────────────
        m = step_re.search(line)
        if m:
            # если предыдущий step уже набрал все метрики — сохраняем
            if current.get('dev') and current.get('test'):
                current.setdefault('gap', round(current['test'] - current['dev'], 4))
                rows.append(current)
            # начинаем новый шаг
            current = {'step': int(m.group(1))}
            
            keys = [k.strip() for k in m.group(2).split('+')]
            raw_vals = re.findall(r"'[^']*'|[^,]+", m.group(3))
            vals = [v.strip().strip("'") for v in raw_vals]
            for k, v in zip(keys, vals):
                try:
                    current[k] = eval(v)      # превращаем 0.001 → float, 8 → int
                except Exception:
                    current[k] = v            # если это строка без кавычек
    
        # ── 2. «Результаты (DEV):»  ───────────────────────
        if "Результаты (DEV):" in line:
            for j in range(i + 1, len(lines)):
                m = mean_re.search(lines[j])
                if m:
                    current['dev'] = float(m.group(1))
                    break
        
        # ── 3. «Результаты (TEST):» + GAP  ────────────────
        if "Результаты (TEST):" in line:
            for j in range(i + 1, len(lines)):
                m = mean_re.search(lines[j])
                if m:
                    current['test'] = float(m.group(1))
                    break
            for j in range(i + 1, len(lines)):
                g = gap_re.search(lines[j])
                if g:
                    current['gap'] = float(g.group(1))
                    break

    # не забываем «добавить хвост»
    if current.get('dev') and current.get('test'):
        current.setdefault('gap', round(current['test'] - current['dev'], 4))
        rows.append(current)

    df = pd.DataFrame(rows)
    if not df.empty:
        df = df.sort_values('dev', ascending=False)
        if top_n is not None:
            df = df.head(top_n)
        df = df.reset_index(drop=True)
    return df


In [2]:
df = parse_smart_log("C:/Users/Alexandr/Desktop/sampling/10.txt",25)

from IPython.display import display
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 160)

display(df.head(15))

,step,lr,num_transformer_heads,tr_layer_number,optimizer,scheduler_type,warmup_ratio,dev,test,gap
0,37,0.0001,8,4,adam,plateau,0.05,0.5904,0.5756,0.0148
1,38,0.0001,8,4,adam,plateau,0.10,0.5904,0.5756,0.0148
2,47,0.0001,8,5,adam,huggingface_cosine_with_restarts,0.05,0.5877,0.5771,0.0106
3,40,0.0001,8,4,adam,huggingface_cosine_with_restarts,0.10,0.5856,0.5751,0.0105
4,64,0.0001,16,5,adam,huggingface_cosine_with_restarts,0.10,0.5830,0.5705,0.0126
5,45,0.0001,8,5,adam,plateau,0.05,0.5814,0.5825,-0.0011
6,46,0.0001,8,5,adam,plateau,0.10,0.5814,0.5825,-0.0011
7,3,0.0010,8,4,sgd,huggingface_cosine_with_restarts,0.05,0.5810,0.5788,0.0022
8,4,0.0010,8,4,sgd,huggingface_cosine_with_restarts,0.10,0.5801,0.5770,0.0031
9,48,0.0001,8,5,adam,huggingface_cosine_with_restarts,0.10,0.5798,0.5731,0.0066


In [3]:
df = parse_smart_log("C:/Users/Alexandr/Desktop/sampling/20.txt",25)

from IPython.display import display
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 160)

display(df.head(15))

,step,lr,num_transformer_heads,tr_layer_number,optimizer,scheduler_type,warmup_ratio,dev,test,gap
0,56,0.0001,16,4,adam,huggingface_cosine_with_restarts,0.10,0.5891,0.5777,0.0114
1,3,0.0010,8,4,sgd,huggingface_cosine_with_restarts,0.05,0.5853,0.5823,0.0030
2,53,0.0001,16,4,adam,plateau,0.05,0.5845,0.5774,0.0072
3,54,0.0001,16,4,adam,plateau,0.10,0.5845,0.5774,0.0072
4,30,0.0010,16,5,adam,plateau,0.10,0.5839,0.5694,0.0145
5,29,0.0010,16,5,adam,plateau,0.05,0.5839,0.5694,0.0145
6,62,0.0001,16,5,adam,plateau,0.10,0.5838,0.5775,0.0063
7,61,0.0001,16,5,adam,plateau,0.05,0.5838,0.5775,0.0063
8,38,0.0001,8,4,adam,plateau,0.10,0.5834,0.5836,-0.0002
9,37,0.0001,8,4,adam,plateau,0.05,0.5834,0.5836,-0.0002


In [4]:
df = parse_smart_log("C:/Users/Alexandr/Desktop/sampling/30.txt",25)

from IPython.display import display
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 160)

display(df.head(15))

,step,lr,num_transformer_heads,tr_layer_number,optimizer,scheduler_type,warmup_ratio,dev,test,gap
0,64,0.0001,16,5,adam,huggingface_cosine_with_restarts,0.10,0.5812,0.5712,0.0100
1,47,0.0001,8,5,adam,huggingface_cosine_with_restarts,0.05,0.5811,0.5710,0.0101
2,48,0.0001,8,5,adam,huggingface_cosine_with_restarts,0.10,0.5800,0.5722,0.0078
3,20,0.0010,16,4,sgd,huggingface_cosine_with_restarts,0.10,0.5791,0.5724,0.0067
4,27,0.0010,16,5,sgd,huggingface_cosine_with_restarts,0.05,0.5790,0.5746,0.0044
5,19,0.0010,16,4,sgd,huggingface_cosine_with_restarts,0.05,0.5783,0.5700,0.0083
6,4,0.0010,8,4,sgd,huggingface_cosine_with_restarts,0.10,0.5774,0.5654,0.0120
7,54,0.0001,16,4,adam,plateau,0.10,0.5773,0.5697,0.0075
8,53,0.0001,16,4,adam,plateau,0.05,0.5773,0.5697,0.0075
9,38,0.0001,8,4,adam,plateau,0.10,0.5761,0.5737,0.0024


In [5]:
df = parse_smart_log("C:/Users/Alexandr/Desktop/sampling/40.txt",25)

from IPython.display import display
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 160)

display(df.head(15))

,step,lr,num_transformer_heads,tr_layer_number,optimizer,scheduler_type,warmup_ratio,dev,test,gap
0,37,0.0001,8,4,adam,plateau,0.05,0.5827,0.5734,0.0093
1,38,0.0001,8,4,adam,plateau,0.10,0.5827,0.5734,0.0093
2,56,0.0001,16,4,adam,huggingface_cosine_with_restarts,0.10,0.5821,0.5589,0.0232
3,3,0.0010,8,4,sgd,huggingface_cosine_with_restarts,0.05,0.5813,0.5801,0.0012
4,39,0.0001,8,4,adam,huggingface_cosine_with_restarts,0.05,0.5810,0.5716,0.0094
5,17,0.0010,16,4,sgd,plateau,0.05,0.5806,0.5757,0.0049
6,18,0.0010,16,4,sgd,plateau,0.10,0.5806,0.5757,0.0049
7,63,0.0001,16,5,adam,huggingface_cosine_with_restarts,0.05,0.5798,0.5767,0.0031
8,28,0.0010,16,5,sgd,huggingface_cosine_with_restarts,0.10,0.5797,0.5710,0.0087
9,19,0.0010,16,4,sgd,huggingface_cosine_with_restarts,0.05,0.5795,0.5731,0.0065


In [6]:
df = parse_smart_log("C:/Users/Alexandr/Desktop/sampling/50.txt",25)

from IPython.display import display
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 160)

display(df.head(15))

,step,lr,num_transformer_heads,tr_layer_number,optimizer,scheduler_type,warmup_ratio,dev,test,gap
0,56,0.0001,16,4,adam,huggingface_cosine_with_restarts,0.10,0.5919,0.5773,0.0146
1,40,0.0001,8,4,adam,huggingface_cosine_with_restarts,0.10,0.5911,0.5771,0.0140
2,54,0.0001,16,4,adam,plateau,0.10,0.5879,0.5774,0.0105
3,53,0.0001,16,4,adam,plateau,0.05,0.5879,0.5774,0.0105
4,37,0.0001,8,4,adam,plateau,0.05,0.5873,0.5722,0.0152
5,38,0.0001,8,4,adam,plateau,0.10,0.5873,0.5722,0.0152
6,18,0.0010,16,4,sgd,plateau,0.10,0.5861,0.5761,0.0100
7,17,0.0010,16,4,sgd,plateau,0.05,0.5861,0.5761,0.0100
8,19,0.0010,16,4,sgd,huggingface_cosine_with_restarts,0.05,0.5838,0.5826,0.0012
9,62,0.0001,16,5,adam,plateau,0.10,0.5837,0.5732,0.0105


In [7]:
df = parse_smart_log("C:/Users/Alexandr/Desktop/sampling/60.txt",25)

from IPython.display import display
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 160)

display(df.head(25))

,step,lr,num_transformer_heads,tr_layer_number,optimizer,scheduler_type,warmup_ratio,weight_decay,dropout,dev,test,gap
0,27,0.0001,8,5,adam,plateau,0.05,0.0,0.20,0.5850,0.5768,0.0082
1,33,0.0001,8,5,adam,plateau,0.10,0.0,0.20,0.5850,0.5768,0.0082
2,32,0.0001,8,5,adam,plateau,0.10,0.0,0.15,0.5827,0.5732,0.0095
3,26,0.0001,8,5,adam,plateau,0.05,0.0,0.15,0.5827,0.5732,0.0095
4,49,0.0001,16,4,adam,plateau,0.05,0.0,0.10,0.5827,0.5709,0.0119
5,55,0.0001,16,4,adam,plateau,0.10,0.0,0.10,0.5827,0.5709,0.0119
6,36,0.0001,8,5,adam,plateau,0.10,0.1,0.20,0.5820,0.5717,0.0104
7,30,0.0001,8,5,adam,plateau,0.05,0.1,0.20,0.5820,0.5717,0.0104
8,29,0.0001,8,5,adam,plateau,0.05,0.1,0.15,0.5815,0.5719,0.0096
9,35,0.0001,8,5,adam,plateau,0.10,0.1,0.15,0.5815,0.5719,0.0096


In [8]:
df = parse_smart_log("C:/Users/Alexandr/Desktop/sampling/70.txt",25)

from IPython.display import display
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 160)

display(df.head(25))

,step,lr,num_transformer_heads,tr_layer_number,optimizer,scheduler_type,warmup_ratio,weight_decay,dropout,dev,test,gap
0,43,0.0001,8,5,adam,huggingface_cosine_with_restarts,0.10,0.0,0.10,0.5935,0.5771,0.0164
1,91,0.0001,16,5,adam,huggingface_cosine_with_restarts,0.10,0.0,0.10,0.5902,0.5680,0.0221
2,73,0.0001,16,5,adam,plateau,0.05,0.0,0.10,0.5900,0.5675,0.0224
3,79,0.0001,16,5,adam,plateau,0.10,0.0,0.10,0.5900,0.5675,0.0224
4,81,0.0001,16,5,adam,plateau,0.10,0.0,0.20,0.5858,0.5720,0.0138
5,75,0.0001,16,5,adam,plateau,0.05,0.0,0.20,0.5858,0.5720,0.0138
6,80,0.0001,16,5,adam,plateau,0.10,0.0,0.15,0.5856,0.5714,0.0141
7,74,0.0001,16,5,adam,plateau,0.05,0.0,0.15,0.5856,0.5714,0.0141
8,85,0.0001,16,5,adam,huggingface_cosine_with_restarts,0.05,0.0,0.10,0.5848,0.5637,0.0211
9,25,0.0001,8,5,adam,plateau,0.05,0.0,0.10,0.5844,0.5690,0.0154


In [9]:
df = parse_smart_log("C:/Users/Alexandr/Desktop/sampling/80.txt",25)

from IPython.display import display
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 160)

display(df.head(25))

,step,lr,num_transformer_heads,tr_layer_number,optimizer,scheduler_type,warmup_ratio,weight_decay,dropout,dev,test,gap
0,2,0.0001,8,4,adam,plateau,0.05,0.0,0.15,0.5735,0.5675,0.0060
1,8,0.0001,8,4,adam,plateau,0.10,0.0,0.15,0.5735,0.5675,0.0060
2,3,0.0001,8,4,adam,plateau,0.05,0.0,0.20,0.5734,0.5641,0.0094
3,9,0.0001,8,4,adam,plateau,0.10,0.0,0.20,0.5734,0.5641,0.0094
4,75,0.0001,16,5,adam,plateau,0.05,0.0,0.20,0.5723,0.5708,0.0015
5,81,0.0001,16,5,adam,plateau,0.10,0.0,0.20,0.5723,0.5708,0.0015
6,20,0.0001,8,4,adam,huggingface_cosine_with_restarts,0.10,0.0,0.15,0.5715,0.5687,0.0028
7,7,0.0001,8,4,adam,plateau,0.10,0.0,0.10,0.5712,0.5706,0.0006
8,1,0.0001,8,4,adam,plateau,0.05,0.0,0.10,0.5712,0.5706,0.0006
9,74,0.0001,16,5,adam,plateau,0.05,0.0,0.15,0.5711,0.5679,0.0032


In [10]:
df = parse_smart_log("C:/Users/Alexandr/Desktop/sampling/90.txt",25)

from IPython.display import display
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 160)

display(df.head(25))

,step,lr,num_transformer_heads,tr_layer_number,optimizer,scheduler_type,warmup_ratio,weight_decay,dropout,dev,test,gap
0,20,0.0001,8,4,adam,huggingface_cosine_with_restarts,0.10,0.0,0.15,0.5881,0.5713,0.0168
1,57,0.0001,16,4,adam,plateau,0.10,0.0,0.20,0.5846,0.5672,0.0174
2,51,0.0001,16,4,adam,plateau,0.05,0.0,0.20,0.5846,0.5672,0.0174
3,14,0.0001,8,4,adam,huggingface_cosine_with_restarts,0.05,0.0,0.15,0.5833,0.5661,0.0172
4,21,0.0001,8,4,adam,huggingface_cosine_with_restarts,0.10,0.0,0.20,0.5826,0.5699,0.0127
5,15,0.0001,8,4,adam,huggingface_cosine_with_restarts,0.05,0.0,0.20,0.5819,0.5640,0.0179
6,13,0.0001,8,4,adam,huggingface_cosine_with_restarts,0.05,0.0,0.10,0.5806,0.5587,0.0219
7,2,0.0001,8,4,adam,plateau,0.05,0.0,0.15,0.5800,0.5686,0.0113
8,8,0.0001,8,4,adam,plateau,0.10,0.0,0.15,0.5800,0.5686,0.0113
9,7,0.0001,8,4,adam,plateau,0.10,0.0,0.10,0.5795,0.5693,0.0103


In [11]:
df = parse_smart_log("C:/Users/Alexandr/Desktop/sampling/100.txt",25)

from IPython.display import display
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 160)

display(df.head(25))

,step,lr,num_transformer_heads,tr_layer_number,optimizer,scheduler_type,warmup_ratio,weight_decay,dropout,dev,test,gap
0,49,0.0001,16,4,adam,plateau,0.05,0.0,0.10,0.5860,0.5702,0.0158
1,55,0.0001,16,4,adam,plateau,0.10,0.0,0.10,0.5860,0.5702,0.0158
2,56,0.0001,16,4,adam,plateau,0.10,0.0,0.15,0.5827,0.5771,0.0056
3,50,0.0001,16,4,adam,plateau,0.05,0.0,0.15,0.5827,0.5771,0.0056
4,2,0.0001,8,4,adam,plateau,0.05,0.0,0.15,0.5816,0.5781,0.0034
5,8,0.0001,8,4,adam,plateau,0.10,0.0,0.15,0.5816,0.5781,0.0034
6,51,0.0001,16,4,adam,plateau,0.05,0.0,0.20,0.5805,0.5720,0.0085
7,57,0.0001,16,4,adam,plateau,0.10,0.0,0.20,0.5805,0.5720,0.0085
8,7,0.0001,8,4,adam,plateau,0.10,0.0,0.10,0.5793,0.5715,0.0079
9,1,0.0001,8,4,adam,plateau,0.05,0.0,0.10,0.5793,0.5715,0.0079


In [12]:
df = parse_smart_log("C:/Users/Alexandr/Desktop/sampling/bi/10.txt",25)

from IPython.display import display
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 160)

display(df.head(25))

,step,lr,num_transformer_heads,tr_layer_number,optimizer,scheduler_type,warmup_ratio,dropout,dev,test,gap
0,5,0.0001,8,4,adam,huggingface_cosine_with_restarts,0.05,0.10,0.5710,0.5575,0.0135
1,14,0.0001,8,5,adam,huggingface_cosine_with_restarts,0.05,0.15,0.5707,0.5670,0.0037
2,13,0.0001,8,5,adam,huggingface_cosine_with_restarts,0.05,0.10,0.5698,0.5650,0.0048
3,9,0.0001,8,5,adam,plateau,0.05,0.10,0.5691,0.5565,0.0126
4,11,0.0001,8,5,adam,plateau,0.10,0.10,0.5691,0.5565,0.0126
5,27,0.0001,16,5,adam,plateau,0.10,0.10,0.5690,0.5678,0.0011
6,25,0.0001,16,5,adam,plateau,0.05,0.10,0.5690,0.5678,0.0011
7,7,0.0001,8,4,adam,huggingface_cosine_with_restarts,0.10,0.10,0.5671,0.5574,0.0097
8,32,0.0001,16,5,adam,huggingface_cosine_with_restarts,0.10,0.15,0.5664,0.5634,0.0031
9,24,0.0001,16,4,adam,huggingface_cosine_with_restarts,0.10,0.15,0.5663,0.5629,0.0034


In [13]:
df = parse_smart_log("C:/Users/Alexandr/Desktop/sampling/bi/20.txt",25)

from IPython.display import display
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 160)

display(df.head(25))

,step,lr,num_transformer_heads,tr_layer_number,optimizer,scheduler_type,warmup_ratio,dropout,dev,test,gap
0,7,0.0001,8,4,adam,huggingface_cosine_with_restarts,0.10,0.10,0.5804,0.5689,0.0116
1,21,0.0001,16,4,adam,huggingface_cosine_with_restarts,0.05,0.10,0.5791,0.5709,0.0082
2,22,0.0001,16,4,adam,huggingface_cosine_with_restarts,0.05,0.15,0.5739,0.5651,0.0088
3,18,0.0001,16,4,adam,plateau,0.05,0.15,0.5737,0.5679,0.0058
4,20,0.0001,16,4,adam,plateau,0.10,0.15,0.5737,0.5679,0.0058
5,15,0.0001,8,5,adam,huggingface_cosine_with_restarts,0.10,0.10,0.5731,0.5638,0.0092
6,13,0.0001,8,5,adam,huggingface_cosine_with_restarts,0.05,0.10,0.5731,0.5673,0.0058
7,5,0.0001,8,4,adam,huggingface_cosine_with_restarts,0.05,0.10,0.5730,0.5689,0.0041
8,8,0.0001,8,4,adam,huggingface_cosine_with_restarts,0.10,0.15,0.5724,0.5631,0.0093
9,30,0.0001,16,5,adam,huggingface_cosine_with_restarts,0.05,0.15,0.5719,0.5702,0.0017


In [14]:
df = parse_smart_log("C:/Users/Alexandr/Desktop/sampling/bi/30.txt",25)

from IPython.display import display
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 160)

display(df.head(25))

,step,lr,num_transformer_heads,tr_layer_number,optimizer,scheduler_type,warmup_ratio,dropout,dev,test,gap
0,20,0.0001,16,4,adam,plateau,0.10,0.15,0.5696,0.5631,0.0065
1,18,0.0001,16,4,adam,plateau,0.05,0.15,0.5696,0.5631,0.0065
2,19,0.0001,16,4,adam,plateau,0.10,0.10,0.5654,0.5636,0.0017
3,17,0.0001,16,4,adam,plateau,0.05,0.10,0.5654,0.5636,0.0017
4,26,0.0001,16,5,adam,plateau,0.05,0.15,0.5619,0.5594,0.0025
5,28,0.0001,16,5,adam,plateau,0.10,0.15,0.5619,0.5594,0.0025
6,11,0.0001,8,5,adam,plateau,0.10,0.10,0.5617,0.5549,0.0069
7,9,0.0001,8,5,adam,plateau,0.05,0.10,0.5617,0.5549,0.0069
8,15,0.0001,8,5,adam,huggingface_cosine_with_restarts,0.10,0.10,0.5609,0.5547,0.0062
9,27,0.0001,16,5,adam,plateau,0.10,0.10,0.5607,0.5584,0.0023


In [15]:
df = parse_smart_log("C:/Users/Alexandr/Desktop/sampling/bi/40.txt",25)

from IPython.display import display
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 160)

display(df.head(25))

,step,lr,num_transformer_heads,tr_layer_number,optimizer,scheduler_type,warmup_ratio,dropout,dev,test,gap
0,19,0.0001,16,4,adam,plateau,0.10,0.10,0.5822,0.5671,0.0151
1,17,0.0001,16,4,adam,plateau,0.05,0.10,0.5822,0.5671,0.0151
2,2,0.0001,8,4,adam,plateau,0.05,0.15,0.5729,0.5643,0.0086
3,4,0.0001,8,4,adam,plateau,0.10,0.15,0.5729,0.5643,0.0086
4,20,0.0001,16,4,adam,plateau,0.10,0.15,0.5727,0.5601,0.0126
5,18,0.0001,16,4,adam,plateau,0.05,0.15,0.5727,0.5601,0.0126
6,6,0.0001,8,4,adam,huggingface_cosine_with_restarts,0.05,0.15,0.5698,0.5626,0.0072
7,8,0.0001,8,4,adam,huggingface_cosine_with_restarts,0.10,0.15,0.5697,0.5623,0.0074
8,11,0.0001,8,5,adam,plateau,0.10,0.10,0.5691,0.5622,0.0069
9,9,0.0001,8,5,adam,plateau,0.05,0.10,0.5691,0.5622,0.0069


In [16]:
df = parse_smart_log("C:/Users/Alexandr/Desktop/sampling/bi/50.txt",25)

from IPython.display import display
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 160)

display(df.head(25))

,step,lr,num_transformer_heads,tr_layer_number,optimizer,scheduler_type,warmup_ratio,dropout,dev,test,gap
0,19,0.0001,16,4,adam,plateau,0.10,0.10,0.5723,0.5676,0.0047
1,17,0.0001,16,4,adam,plateau,0.05,0.10,0.5723,0.5676,0.0047
2,20,0.0001,16,4,adam,plateau,0.10,0.15,0.5721,0.5691,0.0031
3,18,0.0001,16,4,adam,plateau,0.05,0.15,0.5721,0.5691,0.0031
4,30,0.0001,16,5,adam,huggingface_cosine_with_restarts,0.05,0.15,0.5707,0.5654,0.0053
5,21,0.0001,16,4,adam,huggingface_cosine_with_restarts,0.05,0.10,0.5701,0.5630,0.0071
6,29,0.0001,16,5,adam,huggingface_cosine_with_restarts,0.05,0.10,0.5700,0.5695,0.0005
7,24,0.0001,16,4,adam,huggingface_cosine_with_restarts,0.10,0.15,0.5695,0.5629,0.0066
8,27,0.0001,16,5,adam,plateau,0.10,0.10,0.5678,0.5712,-0.0034
9,25,0.0001,16,5,adam,plateau,0.05,0.10,0.5678,0.5712,-0.0034


In [17]:
df = parse_smart_log("C:/Users/Alexandr/Desktop/sampling/5862_адам лучший.txt",25)

from IPython.display import display
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 160)

display(df.head(25))

,step,optimizer,lr,weight_decay,num_transformer_heads,tr_layer_number,hidden_dim,out_features,dev,test,gap
0,21,adam,0.00010,0.00,16,5,256,256,0.5862,0.5820,0.0043
1,262,lion,0.00001,0.01,8,5,256,512,0.5847,0.5728,0.0120
2,226,lion,0.00001,0.00,8,5,256,512,0.5847,0.5730,0.0117
3,128,adam,0.00001,0.01,16,4,512,512,0.5845,0.5731,0.0114
4,92,adam,0.00001,0.00,16,4,512,512,0.5840,0.5768,0.0072
5,104,adam,0.00001,0.00,32,4,512,512,0.5834,0.5743,0.0091
6,5,adam,0.00010,0.00,8,4,256,256,0.5834,0.5740,0.0094
7,127,adam,0.00001,0.01,16,4,512,256,0.5833,0.5759,0.0074
8,80,adam,0.00001,0.00,8,4,512,512,0.5826,0.5739,0.0087
9,139,adam,0.00001,0.01,32,4,512,256,0.5820,0.5736,0.0085


In [18]:
df = parse_smart_log("C:/Users/Alexandr/Desktop/sampling/sgd_2.txt",25)

from IPython.display import display
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 160)

display(df.head(25))

,step,lr,momentum,num_transformer_heads,tr_layer_number,hidden_dim,out_features,dev,test,gap
0,5,0.0010,0.5,16,4,256,256,0.5862,0.5841,0.0021
1,8,0.0010,0.9,8,4,256,256,0.5857,0.5682,0.0175
2,23,0.0001,0.9,16,4,256,256,0.5807,0.5764,0.0043
3,2,0.0010,0.5,8,4,256,256,0.5801,0.5707,0.0094
4,11,0.0010,0.9,16,4,256,256,0.5791,0.5732,0.0059
5,20,0.0001,0.9,8,4,256,256,0.5750,0.5738,0.0013
6,24,0.0001,0.9,16,5,256,256,0.5699,0.5715,-0.0016
7,3,0.0010,0.5,8,5,256,256,0.5696,0.5705,-0.0009
8,10,0.0010,0.9,16,3,256,256,0.5688,0.5594,0.0094
9,12,0.0010,0.9,16,5,256,256,0.5672,0.5726,-0.0054
